# Low Level Tensorflow 

## Introduction

first the imports

In [10]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf



Terminology
* Tensor: central (smallest?) unit in tensorflow. A tensor is represented by an array of any number of dimensions 
* Rank: the number of dimensions a tensor has.
* Shape: A tuple of integers 

e.g. a tensor represented by  2 * 3  array. indicates a Rank-2 tensor (2 dimension). The shape of the tensor would be (2,3).
[[1, 2, 3], [4, 5, 6]]
(1 2 3)
(4 5 6)


In [15]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0)
sum = a + b
multiplication = a * b
division = a / b
print(a)
print(b)
print(sum)
print(multiplication)
print(division)

Tensor("Const_11:0", shape=(), dtype=float32)
Tensor("Const_12:0", shape=(), dtype=float32)
Tensor("add_5:0", shape=(), dtype=float32)
Tensor("mul_2:0", shape=(), dtype=float32)
Tensor("truediv_2:0", shape=(), dtype=float32)


In [22]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())


In [23]:
sess = tf.Session()
print(sess.run(total))

7.0


In [24]:
print(sess.run({'ab':(a,b), 'total': total, 'mult': multiplication, 'trueDiv': division}))

{'ab': (3.0, 4.0), 'total': 7.0, 'mult': 12.0, 'trueDiv': 0.75}


In [21]:
writer.flush()

    #### An example of Random Values

In [44]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 1
print(sess.run(vec))
print(sess.run(vec))

print(sess.run((out1, out2)))

[0.4353665  0.12567902 0.48000693]
[0.21204722 0.68860495 0.46296203]
(array([1.7259429, 1.0290451, 1.6006452], dtype=float32), array([1.7259429, 1.0290451, 1.6006452], dtype=float32))


#### PlaceHolders
more interesting ...

In [45]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y
print(sess.run(z, feed_dict={x:3, y:4.5}))
print(sess.run(z, feed_dict={x:[1,2], y:[2, 4]}))

7.5
[3. 6.]


#### Datasets 

In [59]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]

slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()


In [60]:
# while True:
#     try:
#         print(sess.run(next_item))
#     except tf.errors.OutOfRangeError:
#         break
i = 0
while i < 4:
    print(sess.run(next_item))
    i +=1

[0 1]
[2 3]
[4 5]
[6 7]


In [62]:
r = tf.random_normal([10,3])
dataset = tf.data.Dataset.from_tensor_slices(my_data)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer)
while True:
  try:
    print(sess.run(next_row))
  except tf.errors.OutOfRangeError:
    break

[0 1]
[2 3]
[4 5]
[6 7]


 #### Laaaayers!!! :)


In [74]:
x = tf.placeholder(tf.float32, shape=[None,3])
model = tf.layers.Dense(units=1)
y = model(x)

init = tf.global_variables_initializer()
sess.run(init)
print(sess.run(y, feed_dict={x:[[7.,8.,9.]]}))

[[-18.016333]]


In [79]:
features = {
    'sales' : [[5], [10], [8], [9]],
    'department' : ['sports', 'sports', 'gardening', 'gardening']
}

department_column = tf.feature_column.categorical_column_with_vocabulary_list(
    'department', ['sports', 'gardening']
)

department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'), 
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)

var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))
print(sess.run(inputs))

[[ 1.  0.  5.]
 [ 1.  0. 10.]
 [ 0.  1.  8.]
 [ 0.  1.  9.]]


Training

In [121]:
input = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
expected_output = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)
sum = input.__add__(input)
linear_model = tf.layers.Dense(units=1)

predictions = linear_model(input)
loss = tf.losses.mean_squared_error(labels=expected_output, predictions=predictions)

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print("Before Optimisation")
print(sess.run(predictions))
print(sess.run(loss))
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

loss_value = 0
for i in range(100):
    _, loss_value = sess.run((train, loss))

print("After Optimisation")
print(sess.run(predictions))
print(loss_value)

print("Expected Output")
print(sess.run(expected_output))

Before Optimisation
[[-0.7695125]
 [-1.539025 ]
 [-2.3085375]
 [-3.07805  ]]
0.24599612
After Optimisation
[[-0.47258702]
 [-1.2290008 ]
 [-1.9854145 ]
 [-2.7418284 ]]
0.08617654
Expected Output
[[ 0.]
 [-1.]
 [-2.]
 [-3.]]


In [122]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())